In [2]:
%load_ext autoreload

%autoreload 2

import os
import datetime as dt

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import models
import encoders
import decoders
from training import train, test, checkpoint
from dataloaders import load_data, create_loaders

WANDB_API_KEY= '52c84ab3f3b5c1f999c7f5f389f5e423f46fc04a'

import wandb
wandb.login() #key=WANDB_API_KEY

cuda


wandb: Currently logged in as: raraz15 (use `wandb login --relogin` to force relogin)


True

In [3]:
#dataset_path = '/projects/bassline_transcription/data/datasets'
dataset_path = '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets'
dataset_name = 'traxsource_0-5000_bassline_representations'
M = 8 


data_params = {'dataset_path': dataset_path,
               'dataset_name': dataset_name,
               'scale_type': 'min',
               'M': M,
               'batch_size': 64}

X = load_data(data_params)

K = X.max()+1 # Number of classes, assumes consecutive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

train_loader, test_loader = create_loaders(X, data_params)

Number of classes: 38
Sequence Length: 64
Number of data points: 3859


In [4]:
encoder_params = {'num_embeddings': K,
                  'embedding_dim': 16, 
                  'hidden_size': 128,
                  'dropout': 0.2,  
                  'num_layers': 4,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

#StackedUnidirLSTMDenseDecoder / StackedUnidirectionalLSTMDecoder
decoder_params = {'input_size': encoder_params['hidden_size'],
                  'output_size': K,
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device}

#StackedUnidirectionalLSTMDecoderwithEmbedding 
"""decoder_params = {'num_embeddings': K,
                  'embedding_dim': 16,
                  'hidden_size': 128,                  
                  'num_layers': encoder_params['num_layers'], 
                  'dropout': 0.2,
                  'batch_size': data_params['batch_size'],
                  'sequence_length': sequence_length,
                  'device':device}"""

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 500}

all_params = {'encoder_params': encoder_params, 'decoder_params':decoder_params, 'train_params':train_params}

encoder = encoders.StackedBidirLSTMEncoder(**encoder_params).to(device)

decoder = decoders.StackedUnidirLSTMDecoderwithEmbedding(**decoder_params).to(device)

model = models.AutoEncoder(encoder, decoder, device).to(device)
print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [29]:
encoder = encoders.StackedBidirLSTMEncoder(**encoder_params).to(device)

#decoder = decoders.StackedUnidirLSTMDecoder(**decoder_params).to(device)
decoder = decoders.StackedUnidirLSTMDenseDecoder(**decoder_params).to(device)

model = models.VanillaAutoEncoder(encoder, decoder).to(device)
print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

[autoreload of models failed: Traceback (most recent call last):
  File "/kuacc/apps/anaconda/3.6/envs/root_env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/kuacc/apps/anaconda/3.6/envs/root_env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/kuacc/apps/anaconda/3.6/envs/root_env/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/kuacc/apps/anaconda/3.6/envs/root_env/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, i

TypeError: __init__() got an unexpected keyword argument 'output_size'

In [28]:
if torch.rand(1) < 3:
    print('lol')

lol


In [10]:
project_name = 'PyTorch_experiments'

model_name = dt.datetime.strftime(dt.datetime.now(),"%d_%m__%H_%M")

with wandb.init(project=project_name, name=model_name, config=all_params, entity='nbg'):

    test_loss = test(model, test_loader, criterion, device)
    wandb.log({'test_loss': test_loss})

    for epoch in tqdm(range(train_params['N_epochs'])):

        train_loss = train(model, train_loader, optimizer, criterion, device)
        test_loss = test(model, test_loader, criterion, device)
        
        wandb.log({'train_loss': train_loss, 'test_loss': test_loss})

        if not (epoch % 5):
            print('Epoch: {}, train_loss:{:.6f}'.format(epoch, train_loss))
            checkpoint(model_name, model, optimizer, epoch)
            
        if not (epoch % 50):
            samples = model.sample(24)
            print('Sample: {}'.format(samples[0]))
            wandb.log({'samples': samples})

    test_loss = test(model, test_loader, criterion, device)
    print('Test Loss After Training: {:.6f}'.format(test_loss))
    wandb.log({'test_loss': test_loss})


  0%|          | 1/500 [00:06<57:02,  6.86s/it]

Epoch: 0, train_loss:2.258933
tensor([37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37], device='cuda:0')



  1%|          | 6/500 [00:32<44:22,  5.39s/it]

Epoch: 5, train_loss:1.877414



  2%|▏         | 11/500 [01:01<47:25,  5.82s/it]

...

Epoch: 495, train_loss:0.259663



100%|██████████| 500/500 [48:12<00:00,  5.79s/it]


Test Loss After Training: 1.725295


test_loss,1.72529
_runtime,2905
_timestamp,1621957622
_step,501
train_loss,0.40202


test_loss,█▆▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▆▆▇▇▇█
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁


In [ ]:
model.sample(24)[0]

test_loss = test(model, test_loader, criterion, device)

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss = train(model, train_loader, optimizer, criterion, device)
    
    if epoch+1 % 5:
        print('Epoch: {}, train_loss:{:.6f}'.format(epoch+1, train_loss))

test_loss = test(model, test_loader, criterion, device)
print('Test Loss After Training: {:.6f}'.format(test_loss))

encoder_params = {'num_embeddings': K,
                  'embedding_dim': 16, #??
                  'hidden_size': 128,
                  'dropout': 0,  
                  'num_layers': 4,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

decoder_params = {'input_size': encoder_params['hidden_size'],
                  'output_size': K,
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device}

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 10}

all_params = {'encoder_params': encoder_params, 'decoder_params':decoder_params, 'train_params':train_params}